In [1]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score,balanced_accuracy_score
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
from transformers import pipeline
from datasets import load_dataset
import pandas as pd
import os

In [9]:
dataset = load_dataset("wu981526092/Multidimensional_Multilevel_Bias_Classification")
access_token = "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"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
mps_device = "mps"
local_model = "/Users/zekunwu/Desktop/bias_detector/MD_SL_best_model/checkpoint-7914"
online_model = "holistic-ai/stereotype_detection"
model = pipeline('text-classification', online_model,use_auth_token=access_token,device = mps_device)  # replace with your model name

Found cached dataset csv (/Users/zekunwu/.cache/huggingface/datasets/wu981526092___csv/wu981526092--Multidimensional_Multilevel_Bias_Classification-ca70dd1a187d4f9e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

/Users/zekunwu/Desktop/hallucination_classifier/venv/lib/python3.9/site-packages/transformers/modeling_utils.py:2194: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [10]:
def get_model_predictions(texts):
    # Use model to get predictions for a list of texts
    predictions = model.predict(texts)
    # Convert prediction probabilities to predicted classes
    predicted_classes = [pred['label'] for pred in predictions]
    return predicted_classes

In [11]:
subset = "test"
all_texts = dataset[subset]['text']
all_labels = dataset[subset]['label']
print("Make prediction:")
all_predictions = get_model_predictions(all_texts)

Make prediction:


In [12]:
def process_result(all_predictions,all_labels,data,subset):

    bias_types = ['gender', 'race', 'religion', 'profession']
    index = {}
    for bias_type in bias_types:
            indices = [i for i, b in enumerate(data[subset]['bias_type']) if b == bias_type]
            index[bias_type] = indices
            print(len(index[bias_type]))
    accuracy = {}
    f1 = {}
    precision = {}
    recall = {}
    ba = {}
    counter_other_dimension = {}
    result = {}
    counter = 0
    y_true_overall = []
    y_pred_overall = []
    counter_other_dimension_all = 0
    for bias_type in bias_types:
        y_true = []
        y_pred = []
        other = 0
        for y in index[bias_type]:
            counter += 1
            if all_predictions[y] == "unrelated":
                y_pred.append(2)
                y_pred_overall.append(2)
            elif all_predictions[y] == "counter-stereotype_"+bias_type:
                y_pred.append(1)
                y_pred_overall.append(1)
            elif all_predictions[y] == "stereotype_"+bias_type:
                y_pred.append(0)
                y_pred_overall.append(0)
            else:
                other += 1
                counter_other_dimension_all += 1
                y_pred.append(0)
                y_pred_overall.append(0)
            y_true.append(all_labels[y])
            y_true_overall.append(all_labels[y])
        accuracy[bias_type] = accuracy_score(y_true,y_pred)
        f1[bias_type] = f1_score(y_true,y_pred,average="macro")
        recall[bias_type] = recall_score(y_true,y_pred,average="macro")
        precision[bias_type] = precision_score(y_true,y_pred,average="macro")
        ba[bias_type] = balanced_accuracy_score(y_true,y_pred)
        counter_other_dimension[bias_type] = other

    overall = {}
    overall["accuracy"] = accuracy_score(y_true_overall,y_pred_overall)
    overall["f1"] = f1_score(y_true_overall,y_pred_overall,average="macro")
    overall["recall"] = recall_score(y_true_overall,y_pred_overall,average="macro")
    overall["precision"] = precision_score(y_true_overall,y_pred_overall,average="macro")
    overall["balanced accuracy"] = balanced_accuracy_score(y_true_overall,y_pred_overall)
    overall["count to other dimension"] = counter_other_dimension_all

    result["test size"] = counter
    result["accuracy"] = accuracy
    result["f1"] = f1
    result["recall"] = recall
    result["precision"] = precision
    result["balanced accuracy"] = ba
    result["count to other dimension"] = counter_other_dimension
    result["overall"] = overall

    return result

In [13]:
result = process_result(all_predictions,all_labels,dataset,"test")
result

1316
4717
429
3882


{'test size': 10344,
 'accuracy': {'gender': 0.6261398176291794,
  'race': 0.7000211999152003,
  'religion': 0.7016317016317016,
  'profession': 0.6872746007212777},
 'f1': {'gender': 0.6018740252456125,
  'race': 0.6939920557949955,
  'religion': 0.6993621519499799,
  'profession': 0.6712512845777439},
 'recall': {'gender': 0.6358476992913052,
  'race': 0.7013844914635153,
  'religion': 0.7237344487344487,
  'profession': 0.683407241328775},
 'precision': {'gender': 0.6282548738695999,
  'race': 0.7657898739990866,
  'religion': 0.7120392994801122,
  'profession': 0.6990043129236123},
 'balanced accuracy': {'gender': 0.6358476992913052,
  'race': 0.7013844914635153,
  'religion': 0.7237344487344487,
  'profession': 0.683407241328775},
 'count to other dimension': {'gender': 6,
  'race': 0,
  'religion': 1,
  'profession': 11},
 'overall': {'accuracy': 0.6859048723897911,
  'f1': 0.6737502968306209,
  'recall': 0.6870386894580744,
  'precision': 0.7112483441030376,
  'balanced accuracy

In [14]:
result_df = pd.DataFrame({k: v for k, v in result.items() if k != 'test size' and k != 'overall'})
result_df = result_df.transpose()
print(f"test size: {result['test size']}")
print(result_df)

test size: 10344
                            gender      race  religion  profession
accuracy                  0.626140  0.700021  0.701632    0.687275
f1                        0.601874  0.693992  0.699362    0.671251
recall                    0.635848  0.701384  0.723734    0.683407
precision                 0.628255  0.765790  0.712039    0.699004
balanced accuracy         0.635848  0.701384  0.723734    0.683407
count to other dimension  6.000000  0.000000  1.000000   11.000000


In [15]:
print("Test size: ", result['test size'])
print()

for metric, values in result.items():
    if metric != 'test size':
        print(f"{metric.capitalize()} :")
        for dimension, value in values.items():
            print(f"    {dimension.capitalize()}: {value}")
        print()

Test size:  10344

Accuracy :
    Gender: 0.6261398176291794
    Race: 0.7000211999152003
    Religion: 0.7016317016317016
    Profession: 0.6872746007212777

F1 :
    Gender: 0.6018740252456125
    Race: 0.6939920557949955
    Religion: 0.6993621519499799
    Profession: 0.6712512845777439

Recall :
    Gender: 0.6358476992913052
    Race: 0.7013844914635153
    Religion: 0.7237344487344487
    Profession: 0.683407241328775

Precision :
    Gender: 0.6282548738695999
    Race: 0.7657898739990866
    Religion: 0.7120392994801122
    Profession: 0.6990043129236123

Balanced accuracy :
    Gender: 0.6358476992913052
    Race: 0.7013844914635153
    Religion: 0.7237344487344487
    Profession: 0.683407241328775

Count to other dimension :
    Gender: 6
    Race: 0
    Religion: 1
    Profession: 11

Overall :
    Accuracy: 0.6859048723897911
    F1: 0.6737502968306209
    Recall: 0.6870386894580744
    Precision: 0.7112483441030376
    Balanced accuracy: 0.6870386894580744
    Count to ot